In [77]:
# 3-dimensional forward kinematics simulation for open-chain robot arms with prismatic and/or revolute joints
import numpy as np
import modern_robotics as mr
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import typing
%matplotlib notebook
plt.rcParams['animation.html'] = 'jshtml'

We'll start with a simple 2d representation

In [78]:
# The data we need for our robot arm configuration
link_lengths = np.array([4.0, 5.0, 3.0])
joints = [['r', 'z'], ['r', 'z'], ['r', 'z']] # the first letter defines the joint type and the second letter defines the direction of the screw axis

In [79]:
def generate_link_vectors(lengths: np.ndarray, theta: np.ndarray):
    x_components = lengths * np.cos(theta)
    y_components = lengths * np.sin(theta)

    links = np.stack((x_components, y_components), axis=-1)

    return links

# Testing the link vectors to ensure they have the correct magnitudes and components
theta = np.pi * np.random.rand(3) # generates values in the set [0, pi)
print(theta)

link_vectors = generate_link_vectors(link_lengths, theta)
print(link_vectors)

link_magnitudes = np.sqrt(np.sum(link_vectors ** 2, axis=-1))
link_magnitudes

[2.67291134 1.41239988 1.50761726]
[[-3.5686589   1.80684079]
 [ 0.78867466  4.93740744]
 [ 0.18941113  2.9940146 ]]


array([4., 5., 3.])

In [80]:
# Once we have these individual link vectors, we can find the final position of the end effector using simple vector addition
def calculate_end_effector_position(links: np.ndarray):
    return np.sum(links, axis=0)

calculate_end_effector_position(link_vectors)

array([-2.59057311,  9.73826283])

In [81]:
# # Plotting an animation of the forward kinematics for this 3R planar robot arm
# max_arm_length = np.sum(link_lengths)

# fig, ax = plt.subplots()
# ax.set_xlim(-max_arm_length, max_arm_length)
# ax.set_ylim(-max_arm_length, max_arm_length)

# num_frames = 100
# theta_vals = np.zeros((3, num_frames))
# for value in theta_vals:
#     value = np.linspace(0, np.pi, num_frames)
# print(theta_vals.shape)

# def plot_vector(origin, vector):
#     return ax.plot([origin[0], origin[0]], [vector[0], vector[0]], 'r-', marker='o')
#     # return ax.quiver(origin[0], origin[1], origin[0] + vector[0], origin[0] + vector[1])

# def update(frame):
#     theta = theta_vals[:, frame]
#     link_vectors = generate_link_vectors(link_lengths, theta)
#     link_vectors = link_vectors.tolist()
    
#     for index, vector in enumerate(link_vectors):
#         if index != 0:
#             plot_vector(link_vectors[index - 1], vector)
#         else:
#             plot_vector([0, 0], vector)

# anim = FuncAnimation(fig, update, frames=100)
# plt.show()
# anim.save('result.mp4')

While this method of expressing forward kinematics works in this case, it does not work for all possible 2D arm configurations, and definitely not for 3-dimensional arms. 

For these other arm configurations, we can express the position of the end effector as a homogenous transformation matrix, and apply transformations using screw axis - theta pairs. Here's an example:

In [82]:
# Redefining a more complex version of the robot links, this time with revolute joints with various joint axes, as well as prismatic joints
link_lengths = np.array([4.0, 5.0, 2.0, 2.5])
joints = [['r', 'z'], ['p', 'x'], ['r', 'z'], ['r', 'x']] 

In [83]:
# First we need to generate M, which the matrix representing the home configuration of the robot, with all angles set to zero
# In this case, we'll define the home configuration to be where all the joints are lying on a flat plane, with the last joint facing towards the page in the positive z direction (based on the right hand rule)
M = np.identity(4)
M[0, 3] = np.sum(link_lengths[0:2])
M[2, 3] = link_lengths[-1]
M

array([[1. , 0. , 0. , 9. ],
       [0. , 1. , 0. , 0. ],
       [0. , 0. , 1. , 2.5],
       [0. , 0. , 0. , 1. ]])

In [84]:
# Now we need to define screw axes for each joint on the robot arm
def generate_screw_axis(joint_info: typing.List, q: np.ndarray):
    # note: q defines the position of the joint relative to the origin
    omega = np.zeros(3, dtype=np.float64)
    velocity_axes = {'x': np.array([1, 0, 0]), 'y': np.array([0, 1, 0]), 'z': np.array([0, 0, 1])}

    if joint_info[0] != 'p':
        # as long as the joint isn't prismatic, there will be some element of angular velocity
        omega = velocity_axes[joint_info[1]]
        print("omega: ", omega)
        linear = np.cross(-1 * omega, q)
        print("linear: ", linear)
    else:
        omega = np.array([0, 0, 0])
        print("omega: ", omega)
        linear = velocity_axes[joint_info[1]]
        print("linear: ", linear)

    return np.append(omega, linear)

In [85]:
# With the given screw axis, we can use it with our theta vector to create a skew-symmetric matrix
# that can later be used for the matrix exponential to generate the appropriate transformation matrix and calculate the end-effector position
def calc_end_position(screw_axes: typing.List, theta: np.ndarray, home_config: np.ndarray):
    skew_matrix = lambda x: mr.VecTose3(x)
    screw_matrices = np.array(list(map(skew_matrix, screw_axes)))

    screw_theta_pairs = screw_matrices * theta
    matrix_exp = lambda x: mr.MatrixExp6(x)
    transformation_matrices = np.array(list(map(matrix_exp, screw_theta_pairs)))

    # now that the homogenous transformation matrix has been generated, all we need to do is pre-multiply the matrices in reverse order with the home configuration matrix
    final_config = home_config # initialization

    for transform in reversed(list(transformation_matrices)):
        final_config = np.matmul(transform, final_config)
    
    return final_config

In [86]:
# Generates all the screw axes and combines them into a single array

# First, we need to declare q vectors for each joint
q1 = np.array([link_lengths[0], 0.0, 0.0])
q2 = np.array([sum(link_lengths[:2]), 0.0, 0.0])
q3 = np.array([sum(link_lengths[:3]), 0.0, 0.0])
q4 = np.array([sum(link_lengths[:3]), 0.0, link_lengths[3]])

q1 = np.array([0.0, 0.0, 0.0])
q2 = np.array([link_lengths[0], 0.0, 0.0])
q3 = np.array([sum(link_lengths[:2]), 0.0, 0.0])
q4 = np.array([sum(link_lengths[:3]), 0.0, 0.0])
q_vecs = [q1, q2, q3, q4]
q_vecs

[array([0., 0., 0.]),
 array([4., 0., 0.]),
 array([9., 0., 0.]),
 array([11.,  0.,  0.])]

In [87]:
# Now, we need to map these q vectors, as well as the joint info into our generate screw axes function
screw_axes = list(map(generate_screw_axis, joints, q_vecs))
screw_axes # these are now the calculated screw axes for each joint

omega:  [0 0 1]
linear:  [ 0. -0.  0.]
omega:  [0 0 0]
linear:  [1 0 0]
omega:  [0 0 1]
linear:  [ 0. -9.  0.]
omega:  [1 0 0]
linear:  [ 0.  0. -0.]


[array([ 0.,  0.,  1.,  0., -0.,  0.]),
 array([0, 0, 0, 1, 0, 0]),
 array([ 0.,  0.,  1.,  0., -9.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0., -0.])]

In [91]:
theta = np.array([0.0, np.pi / 4, np.pi / 3, (3 * np.pi) / 4])
# theta = np.pi * np.random.rand(4)
theta

array([0.        , 0.78539816, 1.04719755, 2.35619449])

In [98]:
# returns the transformation matrix for the end position
T = calc_end_position(screw_axes, theta, M)
T

array([[  1.        ,   0.        ,   0.        ,  11.35619449],
       [  0.        ,   0.60947571,  -0.94280904, -23.56277302],
       [  0.        ,   0.70710678,   0.60947571,   1.52368927],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

In [103]:
# now the last step is to apply the transformation matrix and calculate the x, y, and z vectors of the body frame
rotation = T[:3, :3]
translation = T[:3, 3]

x_hat = np.array([1.0, 0.0, 0.0])
y_hat = np.array([0.0, 1.0, 0.0])
z_hat = np.array([0.0, 0.0, 1.0])
space_frame = [x_hat, y_hat, z_hat]

transform = lambda vec: np.dot(rotation, vec)

body_frame = list(map(transform, space_frame))
body_frame, translation # note the last array refers to the new origin of these body frame vectors

([array([1., 0., 0.]),
  array([0.        , 0.60947571, 0.70710678]),
  array([ 0.        , -0.94280904,  0.60947571])],
 array([ 11.35619449, -23.56277302,   1.52368927]))